In [298]:
import pandas as pd
import numpy as np
from tqdm import tqdm

In [317]:
dialogues = pd.read_csv("../data/dialouges_w_map.csv")
characters = pd.read_csv("../data/characters_w_map.csv")


In [318]:
# Initialize a dictionary to store accumulative sentiment and counts
speaker_sentiments = {}

# Create new columns in the dialogues dataframe
dialogues['accumulative_sentiment'] = 0.0
dialogues['average_sentiment'] = 0.0

# Iterate through each row in the dialogues dataframe with a progress bar
for index, row in tqdm(dialogues.iterrows(), total=len(dialogues), desc="Processing dialogues"):
    speaker = row['mapped']
    sentiment_score = row['sentiment_score']
    sentiment_label = row['sentiment_label']
    
    # Convert sentiment_label to a score: 1 for POSITIVE, -1 for NEGATIVE
    sentiment_sign = 1 if sentiment_label == 'POSITIVE' else -1
    
    # Update the sentiment score based on the label
    adjusted_sentiment_score = sentiment_sign * sentiment_score
    
    # Initialize speaker if not present in the dictionary
    if speaker not in speaker_sentiments:
        speaker_sentiments[speaker] = {'accumulative_sentiment': 0, 'sentiment_count': 0, 'sentiment_sign_sum': 0}
    
    # Update accumulative sentiment and sentiment count
    speaker_sentiments[speaker]['accumulative_sentiment'] += adjusted_sentiment_score
    speaker_sentiments[speaker]['sentiment_count'] += 1
    speaker_sentiments[speaker]['sentiment_sign_sum'] += sentiment_sign
    
    # Calculate average sentiment so far
    average_sentiment_so_far = speaker_sentiments[speaker]['sentiment_sign_sum'] / speaker_sentiments[speaker]['sentiment_count']
    
    # Update the dialogues dataframe with the calculated values
    dialogues.at[index, 'accumulative_sentiment'] = speaker_sentiments[speaker]['accumulative_sentiment']
    dialogues.at[index, 'average_sentiment'] = average_sentiment_so_far

# Save the updated dialogues dataframe
dialogues.to_csv("../data/dialouges_over_time.csv", index=False)

print("Updated dialogues dataframe has been saved.")


Processing dialogues: 100%|██████████| 20786/20786 [00:02<00:00, 7787.22it/s]


Updated dialogues dataframe has been saved.


In [356]:
# Count the number of lines for each character
line_counts = dialogues['mapped'].value_counts()

# Get the top 20 characters with the most lines
top_20_characters = line_counts.head(100).reset_index()

# Rename columns for clarity
top_20_characters.columns = ['character', 'line_count']

In [357]:
top_20_characters

,character,line_count
0,tyrion lannister,1752
1,jon snow,1134
2,daenerys targaryen,967
3,cersei lannister,940
4,jaime lannister,931
...,...,...
95,meryn trant,31
96,jory cassel,31
97,anguy,30
98,doran,30


In [358]:
# Select the last row for each unique speaker
end_sentiment_df = dialogues.groupby('mapped').tail(1)

# Create a new DataFrame with only the unique characters and their final scores
end_sentiment = end_sentiment_df[['mapped', 'accumulative_sentiment', 'average_sentiment']].reset_index(drop=True)

# Rename columns for clarity (optional)
end_sentiment.rename(columns={
    'mapped': 'speaker',
    'accumulative_sentiment': 'final_accumulative_sentiment',
    'average_sentiment': 'final_average_sentiment'
}, inplace=True)



In [360]:
df =end_sentiment.loc[end_sentiment['speaker'].isin(top_20_characters['character'])].sort_values(by='final_average_sentiment', ascending=False)
#df = end_sentiment.sort_values(by='final_average_sentiment', ascending=False)

df_dead = []


for index, row in df.iterrows():
    name_map = row['speaker']

    # check if the character is dead
    if pd.isna(characters.loc[characters['mapped']==name_map].Died.values[0]):
        df_dead.append(False)
    else:
        df_dead.append(True)
    
df['dead'] = df_dead


In [361]:
df

,speaker,final_accumulative_sentiment,final_average_sentiment,dead
77,matthos,33.592164,0.400000,True
245,lyanna mormont,31.493314,0.333333,True
153,olyvar,45.697719,0.315789,False
57,doreah,21.353644,0.297297,True
226,tycho,39.858208,0.294118,False
...,...,...,...,...
20,viserys targaryen,-37.713291,-0.343284,True
47,yoren,-40.064173,-0.365854,True
119,rast,-38.219559,-0.419355,True
210,waif,-80.176571,-0.480000,True


In [364]:

df.loc[(df['final_average_sentiment']!=1.0) & (df['final_average_sentiment']!=-1.0)].iloc[:40]

,speaker,final_accumulative_sentiment,final_average_sentiment,dead
77,matthos,33.592164,0.400000,True
245,lyanna mormont,31.493314,0.333333,True
153,olyvar,45.697719,0.315789,False
57,doreah,21.353644,0.297297,True
226,tycho,39.858208,0.294118,False
68,alton,32.279457,0.294118,True
83,ros,80.936190,0.243243,True
197,margaery,232.034402,0.192857,True
167,hizdahr zo loraq,48.293263,0.176471,True
193,mace tyrell,29.663472,0.176471,True


In [342]:
pd.isna(characters.loc[characters['mapped']=='samwell tarly'].Died.values[0])

True

In [327]:
# function that print all dialouge and sentiment for a given speaker
def print_dialouge_and_sentiment(speaker, df=dialogues, column = 'Speaker'):
    df = df.loc[df[column]== speaker]
    for index, row in df.iterrows():
        print(row['Text'], row['sentiment_label'])


In [365]:
print_dialouge_and_sentiment('tycho', df=dialogues, column = 'mapped')

 Welcome to the Iron Bank. Please, sit. POSITIVE
 What can we do for you, Lord Stannis? NEGATIVE
 The Iron Throne is currently occupied by Tommen of the House Baratheon, King of the Andals and the First Men, Lord of the Seven Kingdoms and Protector of the Realm. POSITIVE
 Yes, we have heard this story. POSITIVE
 The king's grandfather tells a different story. A story about a jealous uncle whose attempts to usurp the throne from the rightful king cost the Seven Kingdoms dearly in blood and gold. POSITIVE
 And you feel your blood gives you a claim on our gold? POSITIVE
 Across the Narrow Sea, your books are filled with words like "usurper" and "madman" and "blood right." Here, our books are filled with numbers. We prefer the stories they tell. More plain. Less open to interpretation. How many fighting men remain loyal to you? NEGATIVE
 And how many ships do you have? The ones still afloat, Ser Davos, not at the bottom of Blackwater Bay. NEGATIVE
 And how much wheat and barley and beef an

In [312]:
dialogues

,Text,Speaker,Episode,Season,Show,sentiment_score,sentiment_label,mapped,bracket,accumulative_sentiment,average_sentiment
0,What d’you expect? They’re savages. One lot s...,WAYMAR ROYCE,e1-Winter is Coming,season-01,Game-of-Thrones,2.843596,NEGATIVE,waymar royce,NaN,-2.843596,-1.000000
1,I’ve never seen wildlings do a thing like thi...,WILL,e1-Winter is Coming,season-01,Game-of-Thrones,2.763056,POSITIVE,will,NaN,2.763056,1.000000
2,How close did you get?,WAYMAR ROYCE,e1-Winter is Coming,season-01,Game-of-Thrones,0.582696,NEGATIVE,waymar royce,NaN,-3.426292,-1.000000
3,Close as any man would.,WILL,e1-Winter is Coming,season-01,Game-of-Thrones,2.810488,POSITIVE,will,NaN,5.573544,1.000000
4,We should head back to the wall.,GARED,e1-Winter is Coming,season-01,Game-of-Thrones,3.263553,NEGATIVE,gared,NaN,-3.263553,-1.000000
...,...,...,...,...,...,...,...,...,...,...,...
20781,Uh the Archmaester is less than enthusiastic ...,SAM,e6,season-08,Game-of-Thrones,4.652675,NEGATIVE,sam,NaN,13.813297,-0.012469
20782,"Well, I imagine he isn't using them properly.",BRONN,e6,season-08,Game-of-Thrones,4.785044,NEGATIVE,bronn,NaN,-91.745654,-0.074257
20783,I think we can all agree that ships take prec...,BRIENNE,e6,season-08,Game-of-Thrones,0.476380,POSITIVE,brienne,NaN,-122.387626,-0.158730
20784,I think that's a very presumptuous statement.,BRONN,e6,season-08,Game-of-Thrones,3.653502,POSITIVE,bronn,NaN,-88.092152,-0.071605


In [366]:
df = pd.read_csv("../data/dialouges_over_time.csv")

In [367]:
df

,Text,Speaker,Episode,Season,Show,sentiment_score,sentiment_label,mapped,bracket,accumulative_sentiment,average_sentiment
0,What d’you expect? They’re savages. One lot s...,WAYMAR ROYCE,e1-Winter is Coming,season-01,Game-of-Thrones,2.843596,NEGATIVE,waymar royce,NaN,-2.843596,-1.000000
1,I’ve never seen wildlings do a thing like thi...,WILL,e1-Winter is Coming,season-01,Game-of-Thrones,2.763056,POSITIVE,will,NaN,2.763056,1.000000
2,How close did you get?,WAYMAR ROYCE,e1-Winter is Coming,season-01,Game-of-Thrones,0.582696,NEGATIVE,waymar royce,NaN,-3.426292,-1.000000
3,Close as any man would.,WILL,e1-Winter is Coming,season-01,Game-of-Thrones,2.810488,POSITIVE,will,NaN,5.573544,1.000000
4,We should head back to the wall.,GARED,e1-Winter is Coming,season-01,Game-of-Thrones,3.263553,NEGATIVE,gared,NaN,-3.263553,-1.000000
...,...,...,...,...,...,...,...,...,...,...,...
20781,Uh the Archmaester is less than enthusiastic ...,SAM,e6,season-08,Game-of-Thrones,4.652675,NEGATIVE,samwell tarly,NaN,-52.442450,-0.063291
20782,"Well, I imagine he isn't using them properly.",BRONN,e6,season-08,Game-of-Thrones,4.785044,NEGATIVE,bronn,NaN,-91.745654,-0.074257
20783,I think we can all agree that ships take prec...,BRIENNE,e6,season-08,Game-of-Thrones,0.476380,POSITIVE,brienne,NaN,-122.387626,-0.158730
20784,I think that's a very presumptuous statement.,BRONN,e6,season-08,Game-of-Thrones,3.653502,POSITIVE,bronn,NaN,-88.092152,-0.071605
